We are using BERT to identify and segregate hate speech comments from across the arrays of comments inside YouTube. for which we are using the google built in methods to fetch the required files and comments taken live from the YouTube, comments are fetched on the network of INDIA to define the same.

In [19]:
#All the required libraries and tokens are inititalised
import csv
import time
import torch
import torch.nn as nn
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from transformers import BertTokenizer, BertForSequenceClassification

API's and service fetch are taken here a new .json file created on personal google workshop is used.

Define the necessary variables for YouTube API access, including the scope, API service name, API version, and the path to the client secrets file stored in Google Drive.

In [3]:
# Set up YouTube API credentials
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]
API_SERVICE_NAME = "youtube"
API_VERSION = "v3"
CLIENT_SECRETS_FILE = "client_secret_422487092710-ud2upt20gb8857oc2mdq5vmsn1jake0c.apps.googleusercontent.com.json"

BERT is initialised and token is created to use the sentiment analysis part of the transformer.

Load a pre-trained BERT model and tokenizer for sentiment analysis.
model_name: This constant defines the name of the hate speech detection model.
tokenizer: This object tokenizes the text input.
model: This object is used to perform hate speech detection

In [5]:
# Set up BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


All the required functions for fetch and work are initialised and defined here.

1.Define a function to authenticate with the YouTube API using the OAuth 2.0 flow. This function returns a service object that can be used to make API requests.

2. Define functions for preprocessing text and predicting hate speech using the loaded BERT model.

3. Define a function to fetch live comments from a specified YouTube video using the YouTube API.

4. main execution
In the main part of the script:
Authenticate with the YouTube API.
Fetch live comments from a specified YouTube video.
Save the comments to a CSV file.
Perform hate speech detection on each comment and print the results.

In [7]:
#TOKEN and API authentication
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    credentials = flow.run_local_server(port=7587)
    return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

In [9]:
#Preprocessing of fetched data to give as input to the transformers
def preprocess_text(text):
    tokens = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    return tokens

In [23]:
#Sentiment analysis by the transformer to identify the hate speech comments(1) and non-hate speech comments(0)
def predict_hate_speech(text):
    inputs = preprocess_text(text)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probabilities, dim=-1).item()
    return prediction, probabilities[0][1].item()

#Fetch call for the comments from the YouTube
def fetch_live_comments(youtube_service, video_id, polling_interval=10, max_comments=50):
    comments = []
    total_comments_fetched = 0

    while total_comments_fetched < max_comments:
        response = youtube_service.commentThreads().list(
            part="snippet",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100
        ).execute()

        for item in response.get("items", []):
            comment_text = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(comment_text)

        total_comments_fetched += len(response["items"])
        print(f"Fetched {total_comments_fetched} comments so far.")

        time.sleep(polling_interval)

    return comments
#4
if __name__ == "__main__":
    youtube_service = get_authenticated_service()

    video_id = "P8P_S1Fjl_Q"  # Replace with the actual video ID
    live_comments = fetch_live_comments(youtube_service, video_id)

    csv_filename = "live_comments.csv"
    with open(csv_filename, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Comment"])
        for comment in live_comments:
            writer.writerow([comment])

    print(f"Live comments saved to {csv_filename}")

    # Perform hate speech detection on live comments
    for comment in live_comments:
        prediction, confidence = predict_hate_speech(comment)
        print(f"Comment: {comment}")
        print(f"Prediction: {'Hate speech' if prediction == 1 else 'Not hate speech'}")
        print(f"Confidence: {confidence:.2f}\n")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=422487092710-ud2upt20gb8857oc2mdq5vmsn1jake0c.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A7587%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=ejX36i84e7GhRfEMuCCiTSgbEpJib0&access_type=offline
Fetched 100 comments so far.
Live comments saved to live_comments.csv
Comment: Kya yeti tak ye video pohocha sakte ho?
Prediction: Not hate speech
Confidence: 0.45

Comment: Sport me😢
Prediction: Not hate speech
Confidence: 0.46

Comment: Chadguru🗿🗿🗿🗿🗿🗿
Prediction: Not hate speech
Confidence: 0.47

Comment: 2:02 
😂😅😅😅
Prediction: Not hate speech
Confidence: 0.42

Comment: Perfect 😅
Prediction: Not hate speech
Confidence: 0.44

Comment: ❤❤❤❤😅😅😅
Prediction: Not hate speech
Confidence: 0.47

Comment: 🤣🤣 helerious 8:28
Prediction: Not hate speech
Confidence: 0.39

Comment: Carry bhaiiiiiiiiiii ke aage koi bol skta hai kya 
Munna b